## Satellite Data

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:

    - From MAPBIOMAS:
        - secondary forest age
        - land use land cover
        - fire
    - From ESA CCI Biomass:
        - biomass (Mg C/hectare)

Outputs:

    - From MAPBIOMAS:
        - last observed land use type
        - number of years under each land use type
        - number of fires
        - time since last fire
        - fallow period length



In [3]:
import ee
import geemap
from gee_0_utils import *
initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

age, lulc, fire = desired_lulc()
# biomass for 2020 comes from CCI Biomass
biomass_raw = ee.Image(f"{data_folder}/raw/biomass")

## Biomass - Export secondary

Biomass data is in hectares, but mapbiomas data is 30m resolution.

To deal with edge pixels, we aggregate biomass values to 30m by using the mean (so there is a buffer for land use pixels caught in between two biomass values).

In [7]:
def export_secondary(method, age):
    if method == 'eu':
        # Load the image collections
        transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(roi)
        annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(roi)

        # Define regrowth and degraded conditions
        regrowth = transition.gte(31).And(transition.lte(33))

        # Initialize AgeRegrowth and AgeDegraded
        age = ee.Image.constant(0)

        # Calculate AgeRegrowth
        for i in range(1990, 2020):
            year = 'Dec' + str(i)
            annual_changes_year = annual_changes.select(year)
            condition = annual_changes_year.eq(4).And(regrowth) # were regrowing then AND are regrowing now
            age = age.add(condition.eq(1))

        age = age.selfMask().rename("age")
    biomass = biomass_raw.reduceResolution(reducer = ee.Reducer.mean()) \
        .reproject(crs = age.projection(), scale = 30).toInt16().updateMask(age).rename("biomass")

    secondary = age.addBands([biomass, age.pixelLonLat().float()])

    export_image(secondary, "secondary", folder = method, scale = 30)

export_secondary('mapbiomas', age)

## Remove isolated pixels - Export one_hectare_mask

In the map, there were isolated pixels, often around the edges of forest patches. These would likely be due to misclassification, or follow different behaviors due to edge effects.

To avoid this issue, a kernel is applied here to include only secondary forest patches that are mostly surrounded by other secondary forest pixels.

In [8]:
# convert non-forest pixels from NA to zero
# check what is the most frequent value within each hectare - if it's zero, it means the pixel is surrounded by non-forest cover
one_hectare_mask = age.unmask(0).focalMode(kernelType = "circle", radius = 100, units = "meters").selfMask().rename("one_hectare_mask")

export_image(one_hectare_mask, "one_hectare_mask", folder = "mapbiomas", scale = 30)